In [1]:
import numpy as np



In [9]:
batch_size = 8
n = 20

allNodes = np.random.rand(batch_size, n, 2)

In [20]:
allNodes

array([[[8.31088726e-01, 8.32453744e-01],
        [4.24645079e-01, 8.02142840e-01],
        [6.68960787e-03, 4.21887517e-01],
        [2.23469105e-01, 2.07066924e-01],
        [8.47308828e-01, 7.82561564e-02],
        [9.38871899e-01, 7.20815137e-01],
        [7.54001568e-01, 8.92253214e-01],
        [9.91375941e-01, 2.03454084e-01],
        [8.55953562e-01, 1.83483751e-01],
        [4.09207009e-01, 1.07109201e-01],
        [4.49792410e-01, 3.38902332e-01],
        [7.90242605e-01, 1.94816602e-01],
        [2.49825924e-01, 7.84645019e-01],
        [1.55950963e-01, 5.24030050e-01],
        [2.74854115e-01, 5.06266524e-02],
        [6.58620960e-02, 5.37065598e-01],
        [3.77478262e-01, 2.75919842e-01],
        [3.51902449e-01, 7.89349591e-01],
        [8.70481626e-01, 6.87359704e-02],
        [6.60427446e-01, 2.07631037e-01]],

       [[1.34787942e-01, 8.95110411e-01],
        [5.17808418e-01, 2.91138944e-02],
        [9.11624588e-01, 8.93820867e-01],
        [2.82077185e-01, 8.97605

In [38]:
np.full((2, 3), 4)

array([[4, 4, 4],
       [4, 4, 4]])

In [19]:
def distance_matrix(coords):
    # coords is a numpy array of shape (batch_size, n, 2)
    batch_size, n, _ = coords.shape

    # Repeat the coordinates along a new axis to compute distances between all pairs of coordinates
    coords_expanded = np.repeat(coords[:, np.newaxis, :, :], n, axis=1)
    
    # Compute the pairwise distances between all pairs of coordinates
    distances = np.linalg.norm(coords_expanded - np.transpose(coords_expanded, (0, 2, 1, 3)), axis=-1)

    return distances

In [22]:
T = distance_matrix(allNodes)

array([[ 5, 10, 15,  2,  6,  7, 17, 19, 16,  1, 18, 14,  4,  8, 12,  3,
        11,  9, 13],
       [15,  9, 18,  8,  1,  3,  4,  2,  6, 13, 12, 14, 16, 19,  7, 11,
         5, 10, 17],
       [ 4,  2, 16, 13, 12,  9, 19, 10,  7,  1, 11,  6, 14, 15,  5, 17,
        18,  8,  3],
       [ 8,  2, 14, 19,  1,  5,  6, 10,  9, 11, 17,  4, 16, 12, 13,  7,
         3, 18, 15],
       [12,  1,  8, 19, 13, 16, 18,  2,  5,  6, 14,  4,  3, 17, 10, 11,
        15,  9,  7],
       [ 2, 16,  9,  8, 19,  3, 17, 18, 10,  1, 15, 11,  7,  4, 14, 13,
         6,  5, 12],
       [ 5,  3, 10, 11, 19, 18, 12,  8,  1, 17, 14, 15,  9, 13,  6,  2,
         7, 16,  4],
       [19, 12, 11,  1,  2,  6, 18, 10, 14,  5,  7, 17, 16, 15,  9,  8,
        13,  3,  4]])

In [60]:
n = 19
K = 8
arr = np.tile(np.arange(1, n+1), K).reshape(K,n)
tsp_tour = np.apply_along_axis(np.random.permutation, 1, arr)

In [63]:
tsp_tour[:,0]

array([14, 12, 14, 14,  9, 16, 10, 12])

In [67]:
np.full(3, 2)

array([2, 2, 2])

In [66]:
T[np.arange(K), 0, tsp_tour[:,0]]

array([0.95950536, 0.90115167, 0.75429295, 0.46610985, 0.59583437,
       0.92120284, 0.94102737, 0.06135819])

In [78]:
m = 3
batch_size, n1, _ = allNodes.shape
n = n1 - 1
T = distance_matrix(allNodes)
V = np.full((batch_size, n, m), np.inf)
P = np.full((batch_size, n, m), n+1)
C = np.full((batch_size, n, m), np.inf)
i = 0
r = 0
t = np.zeros(batch_size)
j = i
while j<n:
    if i==j:
        t = T[np.arange(batch_size), 0, tsp_tour[:,j]] * 2
    else:
        t = t - T[np.arange(batch_size), tsp_tour[:,j-1], 0] + T[np.arange(batch_size), tsp_tour[:,j-1], tsp_tour[:,j]] + T[np.arange(batch_size), tsp_tour[:,j], 0]
    V[:, j, 0] = t
    P[:, j, 0] = np.full(batch_size, -1)
    C[:, j, 0] = t
    j += 1

In [94]:
V[:,1,0]

array([1.94436018, 1.814423  , 1.75613019, 2.22139436, 1.33809317,
       1.85332163, 1.93048503, 0.68936411])

In [101]:
len(V[:,0,0] < V[:,1,0])

8

In [96]:
np.maximum(V[:,1,0], V[:,2,0])

array([2.33531206, 3.13021484, 2.63893416, 2.41156545, 2.31250565,
       1.89538588, 2.37739856, 1.83972291])

In [159]:
def SPLIT_mTSP_batch(allNodes, K, tsp_tour):
    batch_size, n1, _ = allNodes.shape
    n = n1 - 1
    T = distance_matrix(allNodes)
    V = np.full((batch_size, n, K), np.inf)
    P = np.full((batch_size, n, K), n+1)
    C = np.full((batch_size, n, K), np.inf)
    i = 0
    r = 0
    t = np.zeros(batch_size)
    j = i
    while j<n:
        if i==j:
            t = T[np.arange(batch_size), 0, tsp_tour[:,j]] * 2
        else:
            t = t - T[np.arange(batch_size), tsp_tour[:,j-1], 0] + T[np.arange(batch_size), tsp_tour[:,j-1], tsp_tour[:,j]] + T[np.arange(batch_size), tsp_tour[:,j], 0]
        V[:, j, 0] = t
        P[:, j, 0] = np.full(batch_size, -1)
        C[:, j, 0] = t
        j += 1
    
    for i in range(1,n):
        for r in range(K-1):
            Current_V = V[:, i, r]
            if Current_V[0] < np.inf:
                t = np.zeros(batch_size)
                j = i
                while j<n:
                    if i==j:
                        t = T[np.arange(batch_size), 0, tsp_tour[:,j]] * 2
                    else:
                        t = t - T[np.arange(batch_size), tsp_tour[:,j-1], 0] + T[np.arange(batch_size), tsp_tour[:,j-1], tsp_tour[:,j]] + T[np.arange(batch_size), tsp_tour[:,j], 0]
                    new_t = np.maximum(t, V[:, i-1, r])
                    updated_samples = new_t < V[:, j, r+1]
                    V[updated_samples, j, r+1] = new_t[updated_samples]
                    P[updated_samples, j, r+1] = np.full(batch_size, i-1)[updated_samples]
                    C[updated_samples, j, r+1] = t[updated_samples]
                    j += 1
    return V[:,n-1, K-1]

In [160]:
V = SPLIT_mTSP_batch(allNodes, 3, tsp_tour)

In [161]:
V

array([4.58182011, 4.54389378, 5.21223712, 3.95226357, 3.69878991,
       4.3951455 , 4.11331123, 3.82549985])

In [168]:
sample = 7
A = "["
for i in range(20):
    A += "["
    A += str(allNodes[sample,i,0])
    A += " "
    A += str(allNodes[sample,i,1])
    A +="]"
    if i < 19:
        A += ";"
A += "]"

A

'[[0.368294195731543 0.6893780577489007];[0.9231286113555104 0.33754666071380357];[0.11828662222505537 0.29322669079923724];[0.5741710693057782 0.7897260260222418];[0.6726048757771038 0.6907195533567281];[0.23170466632589226 0.023817953690009164];[0.9614039056107203 0.8888101842663625];[0.2297634494569698 0.6043669117733065];[0.46410419038559303 0.05614397116541037];[0.6884634336043619 0.4388482294589111];[0.45522202674222145 0.9671638066294278];[0.9118173101225008 0.07808951005527331];[0.31368851099484163 0.6613950871262277];[0.40848598851356777 0.13119468195116757];[0.9057727504704001 0.5463829539438629];[0.017554891018919383 0.7944162789463142];[0.9483207834672926 0.6105854789079275];[0.9869199230219388 0.3190900985631888];[0.16359535441559403 0.33651356493985696];[0.3460103845896648 0.5527048355293837]]'

In [169]:
tsp_tour[sample,:]

array([12, 10,  1,  2, 18,  3, 19,  8,  5, 17, 16,  4, 15, 13,  7,  6,  9,
       11, 14])

In [1]:
pip install ortools

  Attempting uninstall: scipy
    Found existing installation: scipy 1.6.2
    Uninstalling scipy-1.6.2:
      Successfully uninstalled scipy-1.6.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.18.1
    Uninstalling protobuf-3.18.1:
      Successfully uninstalled protobuf-3.18.1
Note: you may need to restart the kernel to use updated packages.
